# Anomaly Detection Testing

This notebook tests the percentage-based anomaly detection algorithm on wandb training data to verify it can detect specific anomalies in training curves.

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
# Load Data from CSV File
df = pd.read_csv('wandb_runs_data.csv')
print(f"Loaded data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

In [ ]:
# Display Data Overview
print("First 5 rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)
print("\nBasic statistics:")
print(df.describe())

In [ ]:
# Check unique runs to find our target runs
print("Unique run names:")
unique_runs = df['run_name'].unique()
for run in sorted(unique_runs):
    print(f"  {run}")

# Look for our specific target runs
target_runs = [
]

print(f"\nTarget runs found:")
for target in target_runs:
    matches = [run for run in unique_runs if target in run]
    print(f"  {target}: {matches}")

In [ ]:
# Implement the percentage-based anomaly detection function
def detect_spikes_and_dips_local(data, spike_threshold=5.0, dip_threshold=5.0, min_distance=1, 
                                prominence_threshold=0.02, smoothing_window=2, local_window=3):
    """Detect spikes and dips using percentage comparison with neighboring points"""
    try:
        import numpy as np
        
        # Convert to numpy array
        values = np.array(data)
        
        if len(values) < 7:  # Need at least 7 points for 3 neighbors + center + 1 buffer
            return {
                'spike_indices': np.array([]),
                'spike_values': np.array([]),
                'dip_indices': np.array([]),
                'dip_values': np.array([]),
                'smoothed_data': values,
                'local_stats': []
            }
        
        # Apply smoothing (default window=2)
        if smoothing_window > 1:
            smoothed = np.convolve(values, np.ones(smoothing_window)/smoothing_window, mode='same')
        else:
            smoothed = values
        
        # Find spikes and dips using percentage comparison
        spike_candidates = []
        dip_candidates = []
        local_stats = []
        
        # Use local_window as the number of neighboring points to compare (default=3)
        neighbors = local_window
        
        for i in range(neighbors, len(smoothed) - neighbors):
            # Get neighboring points (exclude the center point itself)
            left_neighbors = smoothed[i-neighbors:i]
            right_neighbors = smoothed[i+1:i+neighbors+1]
            all_neighbors = np.concatenate([left_neighbors, right_neighbors])
            
            # Calculate average of neighboring points
            neighbor_avg = np.mean(all_neighbors)
            current_value = smoothed[i]
            
            # Store local stats for debugging
            local_stats.append({
                'index': i,
                'current_value': current_value,
                'neighbor_avg': neighbor_avg,
                'neighbors': all_neighbors.tolist()
            })
            
            # Check for spikes: current value is spike_threshold% higher than neighbor average
            if abs(neighbor_avg) > 1e-10:  # Avoid division by zero with very small threshold
                percent_increase = ((current_value - neighbor_avg) / abs(neighbor_avg)) * 100
                if percent_increase >= spike_threshold:
                    spike_candidates.append(i)
            elif current_value > neighbor_avg:  # Handle zero/near-zero case
                spike_candidates.append(i)
            
            # Check for dips: current value is dip_threshold% lower than neighbor average  
            if abs(neighbor_avg) > 1e-10:  # Avoid division by zero with very small threshold
                percent_decrease = ((neighbor_avg - current_value) / abs(neighbor_avg)) * 100
                if percent_decrease >= dip_threshold:
                    dip_candidates.append(i)
            elif current_value < neighbor_avg:  # Handle zero/near-zero case
                dip_candidates.append(i)
        
        # Simple distance filtering - keep all candidates but enforce minimum distance
        spike_indices = []
        if spike_candidates:
            spike_candidates = sorted(spike_candidates)
            spike_indices.append(spike_candidates[0])
            for candidate in spike_candidates[1:]:
                if candidate - spike_indices[-1] >= min_distance:
                    spike_indices.append(candidate)
        
        dip_indices = []
        if dip_candidates:
            dip_candidates = sorted(dip_candidates)
            dip_indices.append(dip_candidates[0])
            for candidate in dip_candidates[1:]:
                if candidate - dip_indices[-1] >= min_distance:
                    dip_indices.append(candidate)
        
        return {
            'spike_indices': np.array(spike_indices),
            'spike_values': values[spike_indices] if spike_indices else np.array([]),
            'dip_indices': np.array(dip_indices),
            'dip_values': values[dip_indices] if dip_indices else np.array([]),
            'smoothed_data': smoothed,
            'local_stats': local_stats
        }
        
    except Exception as e:
        print(f"Error in anomaly detection: {str(e)}")
        return {
            'spike_indices': np.array([]),
            'spike_values': np.array([]),
            'dip_indices': np.array([]),
            'dip_values': np.array([]),
            'smoothed_data': np.array(data),
            'local_stats': []
        }

print("Anomaly detection function loaded successfully!")

In [ ]:
# Test on  train/loss (should have peak around step 430)
target_run = ''
target_metric = 'train/loss'

# Filter data for this specific run and metric
test_data = df[(df['run_name'] == target_run) & (df['metric_name'] == target_metric)].copy()

if len(test_data) > 0:
    print(f"Found {len(test_data)} data points for {target_run} {target_metric}")
    test_data = test_data.sort_values('step')
    
    # Get values and steps
    steps = test_data['step'].values
    values = test_data['value'].values
    
    print(f"Step range: {steps.min()} to {steps.max()}")
    print(f"Value range: {values.min():.6f} to {values.max():.6f}")
    
    # Test anomaly detection
    results = detect_spikes_and_dips_local(values, spike_threshold=5.0, dip_threshold=5.0)
    
    print(f"\nDetected {len(results['spike_indices'])} spikes at steps:")
    for idx in results['spike_indices']:
        step = steps[idx]
        value = values[idx]
        print(f"  Step {step}: {value:.6f}")
    
    print(f"\nDetected {len(results['dip_indices'])} dips at steps:")
    for idx in results['dip_indices']:
        step = steps[idx]
        value = values[idx]
        print(f"  Step {step}: {value:.6f}")
        
    # Check if we found the expected peak around step 430
    expected_step = 430
    found_near_430 = any(abs(steps[idx] - expected_step) <= 20 for idx in results['spike_indices'])
    print(f"\nFound spike near step {expected_step}? {found_near_430}")
    
else:
    print(f"No data found for {target_run} {target_metric}")

In [ ]:
# Test on  (should have dip around step 2330)
target_run2 = ''

# Try to find any metric for this run (might not be train/loss)
available_data = df[df['run_name'] == target_run2]

if len(available_data) > 0:
    print(f"Available metrics for {target_run2}:")
    metrics = available_data['metric_name'].unique()
    for metric in metrics:
        count = len(available_data[available_data['metric_name'] == metric])
        print(f"  {metric}: {count} points")
    
    # Test on the first available metric
    test_metric = metrics[0]
    test_data2 = available_data[available_data['metric_name'] == test_metric].copy()
    test_data2 = test_data2.sort_values('step')
    
    steps2 = test_data2['step'].values
    values2 = test_data2['value'].values
    
    print(f"\nTesting {target_run2} {test_metric}")
    print(f"Step range: {steps2.min()} to {steps2.max()}")
    print(f"Value range: {values2.min():.6f} to {values2.max():.6f}")
    
    # Test anomaly detection
    results2 = detect_spikes_and_dips_local(values2, spike_threshold=5.0, dip_threshold=5.0)
    
    print(f"\nDetected {len(results2['spike_indices'])} spikes at steps:")
    for idx in results2['spike_indices']:
        step = steps2[idx]
        value = values2[idx]
        print(f"  Step {step}: {value:.6f}")
    
    print(f"\nDetected {len(results2['dip_indices'])} dips at steps:")
    for idx in results2['dip_indices']:
        step = steps2[idx]
        value = values2[idx]
        print(f"  Step {step}: {value:.6f}")
        
    # Check if we found the expected dip around step 2330
    expected_step2 = 2330
    found_near_2330 = any(abs(steps2[idx] - expected_step2) <= 50 for idx in results2['dip_indices'])
    print(f"\nFound dip near step {expected_step2}? {found_near_2330}")
    
else:
    print(f"No data found for {target_run2}")

In [ ]:
# Visualize the first test case with plotly
if len(test_data) > 0:
    fig = go.Figure()
    
    # Plot original data
    fig.add_trace(go.Scatter(
        x=steps,
        y=values,
        mode='lines',
        name='Original Data',
        line=dict(color='blue')
    ))
    
    # Plot smoothed data
    fig.add_trace(go.Scatter(
        x=steps,
        y=results['smoothed_data'],
        mode='lines',
        name='Smoothed Data',
        line=dict(color='lightblue', dash='dash')
    ))
    
    # Plot detected spikes
    if len(results['spike_indices']) > 0:
        spike_steps = steps[results['spike_indices']]
        spike_values = values[results['spike_indices']]
        fig.add_trace(go.Scatter(
            x=spike_steps,
            y=spike_values,
            mode='markers',
            name='Detected Spikes',
            marker=dict(color='red', size=10, symbol='triangle-up')
        ))
    
    # Plot detected dips
    if len(results['dip_indices']) > 0:
        dip_steps = steps[results['dip_indices']]
        dip_values = values[results['dip_indices']]
        fig.add_trace(go.Scatter(
            x=dip_steps,
            y=dip_values,
            mode='markers',
            name='Detected Dips',
            marker=dict(color='green', size=10, symbol='triangle-down')
        ))
    
    # Highlight the expected area around step 430
    fig.add_vline(x=430, line_dash="dot", line_color="orange", 
                  annotation_text="Expected Peak ~430")
    
    fig.update_layout(
        title=f'{target_run} {target_metric} - Anomaly Detection Test',
        xaxis_title='Step',
        yaxis_title='Value',
        hovermode='x unified'
    )
    
    fig.show()
else:
    print("No data to visualize")